# Pythonのdataclassについて


## アジェンダ
- dataclassとは
  - 基本機能
- 色々ハマった事
- 参考
  - https://docs.python.org/ja/3/library/dataclasses.html

## dataclassとは
- Python3.7で導入
- その名の通り、データを格納するためのクラス
- よくある `Dict[str, Any]` でデータを持ちまわす時に使える

### 良い所
- `@dataclass` と書くのでデータ格納用であることが一目瞭然
- データ定義のみで欲しい機能が自動で定義される
- dictからの相互変換が簡単
- イミュータブルの設定が簡単

In [1]:
# dataclassesパッケージにいます
from dataclasses import dataclass

In [2]:

# 基本的な定義の方法
@dataclass
class A:
    b: str
    c: str


In [3]:

# 名前付き引数でも、そうでなくても利用可能（名前付きを使った方が良いとは思う）
instance1 = A(b='bb', c='cc')
assert instance1.b == 'bb'
assert instance1.c == 'cc'

instance2 = A('bb', 'cc')
assert instance2.b == 'bb'
assert instance2.c == 'cc'


In [4]:
# 比較も可能
assert instance1 == instance2

instance3 = A('xx', 'yy')
assert instance1 == instance3

AssertionError: 

In [5]:
# dictからの相互変換も可能
d = {'b': 'bb', 'c': 'cc'}
instance4 = A(**d)
assert instance4 == A(b='bb', c='cc')

from dataclasses import asdict
assert asdict(instance4) == {'b': 'bb', 'c': 'cc'}


In [6]:
# イミュータブル
# イミュータブルにする事で、中身が変更されないという事が保証されるので心理的安全性が高い
# dataclassを使用する場合、基本イミュータブルのではないかと思っている（私見です）
@dataclass(frozen=True)
class AA:
    b: str
    c: str
instance5 = AA('b', 'c')
instance5.b = 'zz'

FrozenInstanceError: cannot assign to field 'b'

In [7]:
# 継承も可能
# （詳細は失念したが、実際には断念したので落とし穴があったと思う）
@dataclass(frozen=True)
class AA:
    b: str
    c: str

@dataclass(frozen=True)
class X(AA):
    d: str

X('a', 'b', 'c')


X(b='a', c='b', d='c')

## ハマった事

In [8]:
# 初期化時に受け取った値で別の値を作りたい
@dataclass
class AAA:
    b: str
    c: str

    def __post_init__(self):
        self.d = f'{self.b}_{self.c}'
instance6 = AAA('b', 'c')
assert instance6.d == 'b_c'

In [9]:
# frozenでやりたい
# エラーになる（__post_init__でも既にfrozenが有効になっている）
@dataclass(frozen=True)
class AAAA:
    b: str
    c: str

    def __post_init__(self):
        self.d = f'{self.b}_{self.c}'
AAAA('b', 'c')


FrozenInstanceError: cannot assign to field 'd'

In [10]:
# frozenでやりたい
# object.__setattr__を使用する
# https://docs.python.org/ja/3/library/dataclasses.html#frozen-instances
@dataclass(frozen=True)
class AAAAA:
    b: str
    c: str

    def __post_init__(self):
        object.__setattr__(self, 'd', f'{self.b}_{self.c}')
instance7 = AAAAA('b', 'c')
assert instance7.d == 'b_c'
